In [20]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import paired_distances
import pandas as pd
import numpy as np

In [21]:
documents=["One ring to rule them all.",    #doc0
           "One ring to find them.",        #doc1
           "One Ring to bring them all.",   #doc2
           "And in the darkness bind them."] #doc3

In [22]:
vectorizer = CountVectorizer() #Initiates the vecotrizer 
matrix = vectorizer.fit_transform(documents) #Build the document term matrix. Rows are docs, cols are word frequencies. WARNING!! THE SVD IS DEFINED IN TERMS OF THE TRANSPOSE OF THIS MATRIX!! 
dictionary = vectorizer.get_feature_names() #List of words in matrix

### A vectorizer counts all the words in a set of documents and encodes them in a matrix, $A$.

In [23]:
pd.DataFrame(matrix.todense(),columns=dictionary,index=[f"doc {i}" for i in range(len(documents))]) #WARNING!! THE DOCUMENT TERM MATRIX IS DEFINED AS THE TRANSPOSE OF THIS MATRIX!! 

,all,and,bind,bring,darkness,find,in,one,ring,rule,the,them,to
doc 0,1,0,0,0,0,0,0,1,1,1,0,1,1
doc 1,0,0,0,0,0,1,0,1,1,0,0,1,1
doc 2,1,0,0,1,0,0,0,1,1,0,0,1,1
doc 3,0,1,1,0,1,0,1,0,0,0,1,1,0


### The correlation matrix $C$, is defined $C=A^TA$. The element $C_{ij}=A_{col\,i}^T\circ A_{col\,j}$ relates the $i^{th}$ and $j^{th}$ document with a scalar.

In [24]:
pd.DataFrame(np.dot(matrix.todense(),np.transpose(matrix.todense())),columns=[f"doc {i}" for i in range(len(documents))],index=[f"doc {i}" for i in range(len(documents))])

,doc 0,doc 1,doc 2,doc 3
doc 0,6,4,5,1
doc 1,4,5,4,1
doc 2,5,4,6,1
doc 3,1,1,1,6


### Singular Value Decomposistion (SVD) is a way of approximating a large matrix with a smaller matrix (dimension reduction).
SVD decomposes a matrix into three matricies which can be used to build an approximate version of $A$
\begin{align}
A=&\:T\:S\:D^T\\
A\approx &\: T_{m\times r}\: S_{m\times r}\: (D_{m\times r})^T
\end{align}
Where $r<m$. $A$ has dimension $n\times m$ ($n$ words in $m$ documents). $T$ has dimension $n\times n$. $S$ has dimension $n\times m$. $D$ has dimension $m\times m$. $T$ and $D$ are unitary matricies, and $S$ is a rectangular diagonal matrix.

In [25]:
T, S, D = np.linalg.svd(np.transpose(matrix.todense()), full_matrices=False)

In [26]:
pd.DataFrame(np.around(np.transpose(np.dot(np.dot(T,np.diag(S)),D)),2),columns=dictionary,index=[f"doc {i}" for i in range(len(documents))])

,all,and,bind,bring,darkness,find,in,one,ring,rule,the,them,to
doc 0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0
doc 1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0
doc 2,1.0,-0.0,-0.0,1.0,-0.0,0.0,-0.0,1.0,1.0,0.0,-0.0,1.0,1.0
doc 3,0.0,1.0,1.0,-0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0


### We can then approximate the doc term matrix by using truncated versions of $T$, $S$, and $D$.

Note that, as if by magic, the dimensions are the same as the original doc term matrix!

In [37]:
firstN=2
subT = T[:,:firstN]
subS=np.diag(S)[:firstN,:firstN]
subD=np.transpose(D)[:,:firstN]

pd.DataFrame(np.around(np.dot(np.dot(subT,subS),np.transpose(subD)),decimals=2)).T

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.74,-0.01,-0.01,0.37,-0.01,0.31,-0.01,1.05,1.05,0.37,-0.01,1.04,1.05
1,0.62,0.03,0.03,0.31,0.03,0.26,0.03,0.88,0.88,0.31,0.03,0.91,0.88
2,0.74,-0.01,-0.01,0.37,-0.01,0.31,-0.01,1.05,1.05,0.37,-0.01,1.04,1.05
3,-0.02,1.00,1.00,-0.01,1.00,0.03,1.00,0.00,0.00,-0.01,1.00,1.00,0.00


## The correlation matrix can be calculated with the components of $A$
\begin{align}
C =&\: A^T\: A \\
  =&\:(T\:S\:D^T)^T\:T\:S\:D^T \\
  =&\: D\:S\:T^T\:T\:S\:D^T \\
  =&\: D\:S^2\,D^T \\
\end{align}
Explicitly, $D$ contains the eignenvectors of $C$ with eigenvectors $S^2$.

In [28]:
A=np.dot(np.transpose(D),np.diag(S))
B=np.dot(np.diag(S),D)
C=np.dot(A,B)
pd.DataFrame(np.around(C,decimals=1),columns=[f"doc {i}" for i in range(len(documents))],index=[f"{documents[i]} (doc {i})" for i in range(len(documents))])

,doc 0,doc 1,doc 2,doc 3
One ring to rule them all. (doc 0),6.0,4.0,5.0,1.0
One ring to find them. (doc 1),4.0,5.0,4.0,1.0
One Ring to bring them all. (doc 2),5.0,4.0,6.0,1.0
And in the darkness bind them. (doc 3),1.0,1.0,1.0,6.0


### The correlation matrix can be approximated with the components of $A$
\begin{align}
C \approx&\: D_{m\times r}\:S_{r\times r}^2\:(D_{m\times r})^T \\
\end{align}
Ignoring low order dimensions can improve correlation between similar documents. They will likely be associated with words that are not very common or exist in every document.

In [29]:
firstN=2
subS=np.diag(S)[:firstN,:firstN]
subD=np.transpose(D)[:,:firstN]

A=np.dot(subD,subS)
B=np.dot(subS,np.transpose(subD))
C=np.dot(A,B)
pd.DataFrame(np.around(C,decimals=2),columns=[f"doc {i}" for i in range(len(documents))],index=[f"{documents[i]} (doc {i})" for i in range(len(documents))])

,doc 0,doc 1,doc 2,doc 3
One ring to rule them all. (doc 0),5.29,4.49,5.29,0.98
One ring to find them. (doc 1),4.49,3.83,4.49,1.04
One Ring to bring them all. (doc 2),5.29,4.49,5.29,0.98
And in the darkness bind them. (doc 3),0.98,1.04,0.98,6.00


### We can look at the $T$ matrix to understand what words are strongly associated with each dimension.

In [30]:
pd.DataFrame(abs(T),index=dictionary).sort_values(0,ascending=True)

,0,1,2,3
darkness,0.050475,0.412063,0.024259,1.154219e-17
in,0.050475,0.412063,0.024259,1.154219e-17
the,0.050475,0.412063,0.024259,1.154219e-17
bind,0.050475,0.412063,0.024259,1.154219e-17
and,0.050475,0.412063,0.024259,1.046898e-17
find,0.132125,0.030875,0.680289,6.338383e-16
bring,0.154625,0.054065,0.286689,7.071068e-01
rule,0.154625,0.054065,0.286689,7.071068e-01
all,0.309250,0.108131,0.573378,1.663099e-16
one,0.441375,0.139005,0.106911,4.422174e-17


### We can find the correlation between a new document and an existing SVD.

Say we have a new document we'd like to compare to those already in our list of documents. You do not need to repeat the SVD! First compose the new document into a new document term matrix/vector $q$ with dimensions $n\times1$ (using the origianl vocabulary). Here $n$ is the number of words in our original document term matrix $A$. We can cast $q$ into the reduced subsace by performing
$$q'_{r\times 1}=T_{n\times r}^T \: q$$
The correletion between q and the other documents can then be found be performing
$$C'=D_{m\times r}\: S_{r\times r}\: q'_{r\times 1}$$

In [31]:
dictionaryMap={}
for i in range(len(dictionary)):
    dictionaryMap[dictionary[i]]=i

In [32]:
newDocument = ["Ring! Ring! Banana phone!"]
subVectorizer = CountVectorizer(vocabulary=dictionaryMap)
subMatrix = subVectorizer.fit_transform(newDocument) #rows are docs, cols are word frequencies 
pd.DataFrame(subMatrix.todense(),columns=dictionary,index=newDocument) #WARNING!! THE DOCUMENT TERM MATRIX IS DEFINED AS THE TRANSPOSE OF THIS MATRIX!!

,all,and,bind,bring,darkness,find,in,one,ring,rule,the,them,to
Ring! Ring! Banana phone!,0,0,0,0,0,0,0,0,2,0,0,0,0


In [33]:
print(np.shape(np.transpose(subMatrix.todense())))
reducedSubMatrix=np.dot(np.transpose(T[:,:firstN]),np.transpose(subMatrix.todense()))
pd.DataFrame(reducedSubMatrix) #WARNING!! THE DOCUMENT TERM MATRIX IS DEFINED AS THE TRANSPOSE OF THIS MATRIX!!

(13, 1)


,0
0,-0.882749
1,0.278010


In [34]:
A=np.dot(subD,subS)
subCorrelation = np.dot(A,reducedSubMatrix)
pd.DataFrame(subCorrelation,columns=newDocument,index=[f"{documents[i]} (doc {i})" for i in range(len(documents))])

,Ring! Ring! Banana phone!
One ring to rule them all. (doc 0),2.097649
One ring to find them. (doc 1),1.768288
One Ring to bring them all. (doc 2),2.097649
And in the darkness bind them. (doc 3),0.008263


### The cosine of two vectors is essentially a normalized dot product.

So instead of looking at the correlation matrix, we can instead look at the cosine between different documents to view similarty on a more normalized basis.

$$cos(q,v)=\frac{q\circ v}{\left|q\right|\:\left|v\right|}$$

In [15]:
def cosine(q,v):
    if len(q)==len(v):
        return sum([q[i]*v[i] for i in range(len(q))])/(np.linalg.norm(q)*np.linalg.norm(v))
    else:
        raise ValueError("vectors q and v of unequal length")

In [16]:
def cosineDot(A,B):
    """
    Designed to work with numpy's truly absurd and understandbly deprecated matrix class.
    """
    aDim = np.shape(A)
    bDim = np.shape(B)
    matrix = np.zeros((aDim[0],bDim[1]))
    for i in range(aDim[0]):
        for j in range(bDim[1]):
            matrix[i,j]=cosine(A[i,:].tolist()[0],B[:,j].reshape(-1).tolist()[0])
    return matrix

### Here are the "normalized" results for the *Ring! Ring! Banana phone!* example.

In [17]:
pd.DataFrame(cosineDot(A,reducedSubMatrix),columns=newDocument,index=[f"{documents[i]} (doc {i})" for i in range(len(documents))])

,Ring! Ring! Banana phone!
One ring to rule them all. (doc 0),8.595865e-01
One ring to find them. (doc 1),9.416298e-01
One Ring to bring them all. (doc 2),8.595865e-01
And in the darkness bind them. (doc 3),-3.205961e-16


### Here is the "normalized" correlation matrix (though it's not technically a correlation matrix anymore) for the original SVD example.

In [18]:
firstN=2
subS=np.diag(S)[:firstN,:firstN]
subD=np.transpose(D)[:,:firstN]

A=np.dot(subD,subS)
B=np.dot(subS,np.transpose(subD))
C=cosineDot(A,B)
pd.DataFrame(C,columns=[f"doc {i}" for i in range(len(documents))],index=[f"{documents[i]} (doc {i})" for i in range(len(documents))])

,doc 0,doc 1,doc 2,doc 3
One ring to rule them all. (doc 0),1.000000,0.999033,1.000000,0.174365
One ring to find them. (doc 1),0.999033,1.000000,0.999033,0.217491
One Ring to bring them all. (doc 2),1.000000,0.999033,1.000000,0.174365
And in the darkness bind them. (doc 3),0.174365,0.217491,0.174365,1.000000
